
# Decision Trees with shrinking capability - Classification example


Decision Trees with shrinking capability from SpKit (Classification example)

In this notebook, we show the capability of decision tree from ***spkit*** to analysie the training and testing performace at each depth of a trained tree. After which, a trained tree can be shrink to any smaller depth, ***without retraining it***. So, by using Decision Tree from ***spkit***, you could choose a very high number for a **max_depth** (or just choose -1, for infinity) and analysis the parformance (accuracy, mse, loss) of training and testing (practically, a validation set) sets at each depth level. Once you decide which is the right depth, you could shrink your trained tree to that layer, without explicit training it again to with new depth parameter.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import spkit
print('spkit version :', spkit.__version__)

# just to ensure the reproducible results
np.random.seed(100) 

# Classification  - Diabetes Dataset -  binary class

from spkit.ml import ClassificationTree

# Diabetes Dataset
from sklearn.datasets import load_diabetes
data = load_diabetes()
X = data.data
y = 1*(data.target>np.mean(data.target))

feature_names = data.feature_names
print(X.shape, y.shape)
Xt,Xs,yt,ys = train_test_split(X,y,test_size =0.3)
print(Xt.shape, Xs.shape,yt.shape, ys.shape)


# Train with max_depth =15, Accuracy, Logloss

model = ClassificationTree(max_depth=15)
model.fit(Xt,yt,feature_names=feature_names)
ytp = model.predict(Xt)
ysp = model.predict(Xs)

ytpr = model.predict_proba(Xt)[:,1]
yspr = model.predict_proba(Xs)[:,1]

print('Depth of trained Tree ', model.getTreeDepth())
print('Accuracy')
print('- Training : ',np.mean(ytp==yt))
print('- Testing  : ',np.mean(ysp==ys))
print('Logloss')
Trloss = -np.mean(yt*np.log(ytpr+1e-10)+(1-yt)*np.log(1-ytpr+1e-10))
Tsloss = -np.mean(ys*np.log(yspr+1e-10)+(1-ys)*np.log(1-yspr+1e-10))
print('- Training : ',Trloss)
print('- Testing  : ',Tsloss)


# Plot Trained Tree

plt.figure(figsize=(15,12))
model.plotTree()


# Analysing the Learning Curve with test data (validation set)

Lcurve = model.getLcurve(Xt=Xt,yt=yt,Xs=Xs,ys=ys,measure='acc')
print(Lcurve)

model.plotLcurve()
plt.xlim([1,model.getTreeDepth()])
plt.xticks(np.arange(1,model.getTreeDepth()+1))
plt.show()


# Learning curve with tree

plt.figure(figsize=(10,8))
model.plotTree(show=False,Measures=True,showNodevalues=True,showThreshold=False)


# Shrinking the trained tree to depth=7

model.updateTree(shrink=True,max_depth=7)


ytp = model.predict(Xt)
ysp = model.predict(Xs)

ytpr = model.predict_proba(Xt)[:,1]
yspr = model.predict_proba(Xs)[:,1]

print('Depth of trained Tree ', model.getTreeDepth())
print('Accuracy')
print('- Training : ',np.mean(ytp==yt))
print('- Testing  : ',np.mean(ysp==ys))
print('Logloss')
Trloss = -np.mean(yt*np.log(ytpr+1e-10)+(1-yt)*np.log(1-ytpr+1e-10))
Tsloss = -np.mean(ys*np.log(yspr+1e-10)+(1-ys)*np.log(1-yspr+1e-10))
print('- Training : ',Trloss)
print('- Testing  : ',Tsloss)


# # Plotting final tree

plt.figure(figsize=(10,6))
model.plotTree(show=False,Measures=True,showNodevalues=True,showThreshold=True)